# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [3]:
DATA_FOLDER = './Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [196]:
# Write your answer here
import os 
import pandas as pd

# Ebola Folder 
DATA_FOLDER_ebola = DATA_FOLDER + '/ebola'
# Directories in the folder
ebola_directories = os.listdir(DATA_FOLDER_ebola)
# Remove License File
del ebola_directories[ebola_directories.index('LICENSE')]

# Names of the columns which contain the total number of daily incidents
data_columns = {'liberia':['Date','Variable', 'National'],
                'sl': ['date','variable', 'National'],
                'guinea':['Date','Description', 'Totals']
               }

### Load All CSV Files Contained In One Folder


In [197]:
# Loads the information relevant to deaths and cases within the csv files contained in the directory for a given country

def load_csv_files_into_dataframe(folder, country):
    country_folder = folder + '/'+ country +'_data'
    csv_files = os.listdir(country_folder)    
    
    all_data = pd.DataFrame(index = ['Date'], columns = [])
    
    totals = data_columns[country][2]
    description = data_columns[country][1]
    date = data_columns[country][0]
    
    for i, file in enumerate(csv_files):
        
        daily_csv = pd.read_csv((country_folder + '/' +file))
        
        # Find columns containing death or case keywords
        motives  = daily_csv[description][daily_csv[description].str.contains("death" or "case")]
        # Create a dictionary with the date and country
        daily_dict = {'Date':daily_csv[date][0], 'Country':country}
        # Add the motives with corresponding total / national values to the dictionary
        daily_dict.update(dict(zip(motives.tolist(), daily_csv[totals][motives.index].tolist())))
        # Append the dictionary to the dataframe
        all_data = all_data.append(daily_dict, ignore_index = True)
    
    all_data = all_data.iloc[1:]
    all_data.set_index(['Date','Country'], inplace = True)
    return all_data

### Liberia Dataset

In [198]:
liberia_data = load_csv_files_into_dataframe(DATA_FOLDER_ebola,'liberia')

liberia_data

('National', 'Variable', 'Date')


,,Cumulative deaths among HCW,Newly Reported deaths in HCW,Newly reported deaths,Total death/s in confirmed cases,"Total death/s in confirmed, probable, suspected cases",Total death/s in probable cases,Total death/s in suspected cases,"Total death/s in confirmed, probable, suspected cases","Total death/s in confirmed, probable, suspected cases"
Date,Country,,,,,,,,,
6/16/2014,liberia,1.0,0.0,2.0,8.0,16.0,6.0,2.0,NaN,NaN
6/17/2014,liberia,1.0,0.0,0.0,8.0,16.0,6.0,2.0,NaN,NaN
6/22/2014,liberia,1.0,0.0,4.0,16.0,25.0,8.0,1.0,NaN,NaN
6/24/2014,liberia,2.0,0.0,4.0,18.0,32.0,8.0,6.0,NaN,NaN
6/25/14,liberia,3.0,1.0,3.0,20.0,37.0,8.0,9.0,NaN,NaN
6/28/2014,liberia,5.0,0.0,1.0,26.0,49.0,10.0,13.0,NaN,NaN
6/29/2014,liberia,5.0,0.0,0.0,26.0,49.0,10.0,13.0,NaN,NaN
7/1/2014,liberia,8.0,1.0,5.0,30.0,61.0,17.0,14.0,NaN,NaN
7/2/2014,liberia,8.0,0.0,5.0,32.0,66.0,20.0,14.0,NaN,NaN


### Guinea Dataset

In [199]:
guinea_data = load_csv_files_into_dataframe(DATA_FOLDER_ebola,'guinea')

guinea_data

('Totals', 'Description', 'Date')


,,New deaths registered today,New deaths registered today (confirmed),New deaths registered today (probables),New deaths registered today (suspects),Number of death of confirmed cases among health workers,Number of deaths of confirmed cases among health workers,Number of deaths of probables cases among health workers,Total deaths (confirmed + probables + suspects),Total deaths of confirmed,Total deaths of probables,Total deaths of suspects,Total of deaths in confirmed cases in CTE,New deaths registered,New deaths registered among health workers,Total deaths registered among health workers
Date,Country,,,,,,,,,,,,,,,
2014-08-04,guinea,2.0,2.0,0.0,0.0,0.0,12.0,8.0,363,228,133,2,138.0,NaN,NaN,NaN
2014-08-26,guinea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,958,292,141,2,NaN,5,0,26
2014-08-27,guinea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,438,294,142,2,NaN,2,0,26
2014-08-30,guinea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,468,324,142,2,NaN,5,0,26
2014-08-31,guinea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,489,337,150,2,NaN,3,0,26
2014-09-02,guinea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,501,349,150,2,NaN,5,0,26
2014-09-04,guinea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,522,368,151,3,NaN,5,0,26
2014-09-07,guinea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,557,402,151,4,NaN,4,0,28
2014-09-08,guinea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,557,402,151,4,NaN,4,0,28


### Sierra Leone Dataset

In [200]:
sl_data = load_csv_files_into_dataframe(DATA_FOLDER_ebola,'sl')

sl_data

('National', 'variable', 'date')


,,death_confirmed,death_probable,death_suspected,etc_cum_deaths,etc_new_deaths
Date,Country,,,,,
2014-08-12,sl,264,34,5,NaN,NaN
2014-08-13,sl,273,34,5,NaN,NaN
2014-08-14,sl,280,34,5,NaN,NaN
2014-08-15,sl,287,34,5,NaN,NaN
2014-08-16,sl,297,34,5,NaN,NaN
2014-08-17,sl,305,34,5,NaN,NaN
2014-08-18,sl,312,34,5,NaN,NaN
2014-08-19,sl,320,34,5,NaN,NaN
2014-08-20,sl,322,34,5,NaN,NaN


# TODO :  
- Format the dates to be the same for all the datasets
- Decide on what is a case and a death for each country -> add new columns to the existing df
- Merge the new columns of the 3 countries with the indexes to create the final df 

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here
DATA_FOLDER_microbiome = DATA_FOLDER + '/microbiome'

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here